## C S 329E HW 7

# Naive Bayes 

## Abe Mankavil Group #22

For this week's homework we are going explore one new classification technique:

  - Naive Bayes

We are reusing the version of the Melbourne housing data set from HW6, to predict the housing type as one of three possible categories:

  - 'h' house
  - 'u' duplex
  - 't' townhouse

In addition to building our own Naive Bayes classifier, we are going to compare the performace of our classifier to the [Gaussian Naive Bayes Classifier](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) available in the scikit-learn library. 

In [226]:
# These are the libraries you will use for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
from sklearn.naive_bayes import GaussianNB # The only thing in scikit-learn you can use this assignment

# Starting off loading a training set and setting a variable for the target column, "Type"
df_melb = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/81b236aecee57f6cf65e60afd865d2bb/raw/56ddb53aa90c26ab1bdbfd0b8d8229c8d08ce45a/melb_data_train.csv')
target_col = 'Type'
df_melb.head()

,Rooms,Type,Price,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt
0,2,h,399000,7/5/16,8.7,3032,1,1.0,904,53.0,1985.0
1,3,h,1241000,28/08/2016,13.9,3165,1,1.0,643,NaN,NaN
2,2,u,550000,8/7/17,3.0,3067,1,1.0,1521,NaN,NaN
3,3,u,691000,24/06/2017,8.4,3072,1,1.0,170,NaN,NaN
4,2,u,657500,19/11/2016,4.6,3122,1,1.0,728,73.0,1965.0


## Q1 - Fix a column of data to be numeric
If we inspect our dataframe, `df_melb` using the `dtypes` method, we see that the column "Date" is an object.  However, we think this column might contain useful information so we want to convert it to [seconds since epoch](https://en.wikipedia.org/wiki/Unix_time). Use only the exiting imported libraries to create a new column "unixtime". Be careful, the date strings in the file might have some non-uniform formating that you have to fix first.  Print out the min and max epoch time to check your work.  Drop the original "Date" column. 

THESE ARE THE EXACT SAME INSTRUCTIONS FROM HW6! Please take this opportunity to reuse your code (if you got it right last time). 

In [227]:
# normalize date accepts the date string as shown in the df_melb 'Date' column,
# and returns a data in a standarized format
def standardize_date(d):
    converted_timestamp = pd.to_datetime(d,dayfirst=True).to_pydatetime()
    timestamp_obj = time.strptime(str(converted_timestamp),'%Y-%m-%d %H:%M:%S')
    return int(time.mktime(timestamp_obj))

In [228]:
df_melb['Date'] = df_melb['Date'].apply( standardize_date )
df_melb['unixtime'] = df_melb['Date']
df_melb = df_melb.drop(columns="Date")


## Q2 Calculating the prior probabilities
Calculate the prior probabilities for each possible "Type" in `df_melb` and populate a dictionary, `dict_priors`, where the key is the possible "Type" values and the value is the prior probabilities. Show the dictionary. Do not hardcode the possible values of "Type".  Don't forget about [value counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html). 

In [229]:
dict_priors = {df_melb['Type'].value_counts().index[i]:(df_melb['Type'].value_counts()[i])/(len(df_melb)) 
               for i in range(len(df_melb['Type'].value_counts()))}

# # show the priors
dict_priors

{'h': 0.452, 'u': 0.418, 't': 0.13}

## Q3 Create a model for the distribution of all of the numeric attributes
For each class, and for each attribute calculate the sample mean and sample standard deviation.  You should store the model in a nested dictionary, `dict_nb_model`, such that `dict_nb_model['h']['Rooms']` is a tuple containing the mean and standard deviation for the target Type 'h' and the attribute 'Rooms'.  Show the model using the `display` function. You should ignore entries that are `NaN` in the mean and [standard deviation](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html) calculation. 

In [230]:
dict_nb_model = dict()
for target in dict_priors.keys():
    df_target = df_melb[df_melb['Type'] == target].drop([target_col], axis=1)
    dict_nb_model[target] = {column:(df_target[column].std(), df_target[column].mean())
                             for column in df_target.columns}

In [231]:
display(dict_nb_model)

{'h': {'Rooms': (0.725826420112775, 3.269911504424779),
  'Price': (586296.5794417894, 1189022.3451327435),
  'Distance': (7.397501132737295, 12.086725663716816),
  'Postcode': (98.35750345419703, 3103.8982300884954),
  'Bathroom': (0.6720871086493074, 1.5619469026548674),
  'Car': (0.932759177140425, 1.7777777777777777),
  'Landsize': (3830.7934157687173, 932.9646017699115),
  'BuildingArea': (54.62662837301433, 156.2433962264151),
  'YearBuilt': (32.4618763471547, 1954.900826446281),
  'unixtime': (13838422.086267864, 1485736247.7876105)},
 'u': {'Rooms': (0.5908453859944255, 2.0430622009569377),
  'Price': (217947.32866736987, 634207.1770334928),
  'Distance': (5.609778714430756, 8.760287081339714),
  'Postcode': (87.18475679946476, 3120.4545454545455),
  'Bathroom': (0.4222815154866222, 1.1818181818181819),
  'Car': (0.47231993860297056, 1.1483253588516746),
  'Landsize': (1394.3403794653257, 436.23444976076553),
  'BuildingArea': (45.95943801516662, 83.85585585585585),
  'YearBuil

## Q4 Write a function that calculates the probability of a Gaussian
Given the mean ($\mu$), standard deviation ($\sigma$), and a observed point, `x`, return the probability.  
Use the formula $p(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$ ([wiki](https://en.wikipedia.org/wiki/Normal_distribution)).  You should use [numpy's exp](https://numpy.org/doc/stable/reference/generated/numpy.exp.html) function in your solution. 

In [232]:
def get_p(mu, sigma, x):
    coeff = 1/(sigma*np.sqrt(np.pi*2))
    exponent = -0.5*(((x-mu)/sigma)**2)
    return coeff*(np.exp(exponent))

In [233]:
# Test it. You should get a value of 0.19333405840142462
p = get_p( 0, 2, 0.5)
p

0.19333405840142462

## Q5 Write the Naive Bayes classifier function
The Naive Bayes classifier function, `nb_class`, should take as a parameter the prior probability dictionary. `dict_priors`, the dictionary containing all of the gaussian distribution information for each attribue, `dict_nb_model`, and a single observation row (a series generated from iterrows) of the test dataframe. It should return a single target classification. For this problem, all of our attributes are numeric and modeled as Gaussians, so we don't worry about categorical data. Make sure to skip attributes that do not have a value in the observation.  Do not hardcode the possible classification types. 

In [234]:
def nb_class(dict_priors, dict_nb_model, observation):
    conditional_prob_dict = {key:dict_priors[key] for key in dict_priors.keys()}
    for type in dict_priors.keys():
        for feature in dict_nb_model[type].keys():
            if pd.notnull(observation[feature]):
                sigma = dict_nb_model[type][feature][0]
                mu = dict_nb_model[type][feature][1]
                x_n = observation[feature]
                conditional_prob_dict[type] *= get_p(mu, sigma, x_n)
    return max(conditional_prob_dict, key=conditional_prob_dict.get)

## Q6 Calculate the accuracy using Naive Bayes classifier function on the test set
Load the test set from file, convert date to unix time and drop the date column, classify each row using your `nb_class`, and then show the accuracy on the test set. 

In [235]:
df_test = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/c3d53303cebbd986b166591d19254bac/raw/94eb3b2d500d5f7bbc0441a8419cd855349d5d8e/melb_data_test.csv')
df_test['Date'] = df_test['Date'].apply( standardize_date )
df_test['unixtime'] = df_test['Date']
df_test = df_test.drop(columns="Date")
df_test.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,3,h,1116000,17.9,3192,1,2.0,610,NaN,NaN,1498885200
1,3,h,2030000,11.2,3186,2,2.0,366,NaN,NaN,1472360400
2,3,h,1480000,10.7,3187,2,2.0,697,143.0,1925.0,1478498400
3,3,u,1203500,12.3,3166,2,2.0,311,127.0,2000.0,1495861200
4,3,h,540000,14.7,3030,2,2.0,353,135.0,2011.0,1504414800


In [236]:
predictions = []
for (indx,row) in df_test.iterrows():
    predictions.append(nb_class(dict_priors, dict_nb_model, row))

In [237]:
def get_acc(s_1, s_2):
    return sum(s_1 == s_2)/len(s_1)

acc = get_acc(np.array(predictions), df_test['Type'])

In [238]:
# You should get and accuracy of 57.00%
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 57.00%


## Use scikit-learn to do the same thing!

Now we understand the inner workings of the Naive Bayes algorithm, let's compare our results to [scikit-learn's Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) implementation. Use the [GaussianNB](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) to train using the `df_melb`dataframe and test using the `df_test` dataframe. Remember to split `df_melb` into a `df_X` with the numerical attributes, and a `s_y` with the target column. On the `df_melb` frame you will have to fill the empty attributes via imputation since the scikit-learn library can not handle missing values.  Use the same method you used in the last homework (filling the training data with the mean of the non-nan values). 

Answer the following in a markdown cell: do you think imputation hurt or helped the classifier?

In [239]:
# Imputation training
dict_imputation = dict()

df_X, s_y  = df_melb.drop(columns=['Type']), df_melb['Type'] 

# Imputation - apply on the test data
for col in df_X.columns:
    dict_imputation[col] = df_X[col].mean()
    df_X[col].fillna(dict_imputation[col],inplace=True)


# Seperate the attributes from the target_col
df_X_test, s_y_test = df_test.drop(columns=['Type']), df_test['Type']
for col in df_X_test.columns:
    df_X_test[col].fillna(dict_imputation[col],inplace=True)

In [240]:
gnb = GaussianNB()
s_y_pred = gnb.fit(df_X, s_y).predict(df_X_test)

In [241]:
acc = get_acc(np.array(s_y_pred), s_y_test)
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 37.00%


## ANSWER TO "do you think imputation hurt or helped the classifier?" 
### The imputation definetly hurt the classifier because the accuracy reduced by nearly 20%